In [ ]:
import pandas as pd
df_proposals = pd.read_excel('proposals.xlsx')

df_proposals['votes'].describe()

count     80338.000000
mean        452.022829
std        6495.491181
min           0.000000
25%           1.000000
50%           4.000000
75%          27.000000
max      213953.000000
Name: votes, dtype: float64

In [ ]:
import re

# 유효한 문자 패턴 정의
valid_pattern = re.compile(r'[a-zA-Z0-9!@#$%^&*()_\-+{}\[\]:;≥"\'<>,.?/\\|`~\s]')


# 유효한 문자만 추출
def get_valid(text):
    if isinstance(text, str): 
        return ''.join(valid_pattern.findall(text))
    return text  


url_pattern = re.compile(
    r'http[s]?://\S+|www\.\S+|ipfs://\S+', re.IGNORECASE
)

def remove_urls_and_ipfs(text):
    if isinstance(text, str): 
        return url_pattern.sub('', text)
    return text  


# 빈 괄호를 탐지하기 위한 정규 표현식 패턴
empty_parentheses_pattern = re.compile(r'\(\s*\)')
def remove_empty_parentheses(text):
    if isinstance(text, str):  # 문자열인지 확인
        return empty_parentheses_pattern.sub('', text)
    return text  # 문자열이 아닌 경우 그대로 반환


wallet_pattern = re.compile(r'\b0x\w+\b', re.IGNORECASE)

# 0x로 시작하는 지갑 주소를 제거하는 함수 정의
def remove_wallet_addresses(text):
    if isinstance(text, str):  # 문자열인지 확인
        return wallet_pattern.sub('', text)
    return text  # 문자열이 아닌 경우 그대로 반환

# 연속된 줄 바꿈 문자를 탐지하기 위한 정규 표현식 패턴
multiple_newlines_pattern = re.compile(r'\n{2,}')

# 연속적인 줄 바꿈 문자를 하나로 변경하는 함수 정의
def replace_multiple_newlines(text):
    if isinstance(text, str):  # 문자열인지 확인
        return multiple_newlines_pattern.sub('\n', text)
    return text  # 문자열이 아닌 경우 그대로 반환




In [ ]:
top20 = ["0xgov.eth",
"1inch.eth",
"aave.eth",
"aavegotchi.eth",
"arbitrumfoundation.eth",
"badgerdao.eth",
"balancer.eth",
"comp-vote.eth",
"cvx.eth",
"curve.eth",
"dydxgov.eth",
"ens.eth",
"frax.eth",
"olympusdao.eth",
"opcollective.eth",
"cakevote.eth",
"radiantcapital.eth",
"suchigov.eth",
"graphprotocol.eth",
"uniswapgovernance.eth"]

In [ ]:
# get valid literals
df_proposals = df_proposals.dropna(subset=['title', 'body'])
for v in ["title", "body"]:
    df_proposals[v] = df_proposals[v] \
                        .apply(get_valid) \
                        .apply(remove_urls_and_ipfs) \
                        .apply(remove_empty_parentheses) \
                        .apply(remove_wallet_addresses) \
                        .apply(replace_multiple_newlines)


# ![]( 제거
df_proposals['body'] = df_proposals['body'].apply(lambda x: x.replace('![](', ''))

# drop empty rows
df_proposals = df_proposals.dropna(subset=['title', 'body'])

# top20에 있는 space만 추출
df_proposals[df_proposals["space_id"].isin(top20)]

# state closed
df_proposals = df_proposals[df_proposals['state'] == 'closed']

# 동일한 proposal 제거
df_proposals = df_proposals.drop_duplicates(subset=['body'])

# votes > 10 
df_proposals = df_proposals[df_proposals['votes'] > 10]

# body 문자열 150자 이상
df_proposals = df_proposals[df_proposals['body'].apply(lambda x: len(x) >= 150)]


In [ ]:
df_proposals.shape
# (17006, 34)


df_proposals['votes'].describe()

count     17006.000000
mean       1870.805774
std       13873.625010
min          11.000000
25%          24.000000
50%          56.000000
75%         173.000000
max      213953.000000
Name: votes, dtype: float64

In [ ]:
del df_proposals["space"]
df_proposals.to_excel('proposals_top20.xlsx', index=False, engine='xlsxwriter')